# K-Vecinos Primer Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


## Datos

In [ ]:
data = pd.read_csv('creditos.csv')
data.head()

## Bueno Vs Malo

In [ ]:
bueno = data[data["cumplio"]==1]
malo = data[data["cumplio"]==0]

# Grafica de los datos
plt.scatter(bueno["edad"], bueno["credito"], marker="*", s=150, color="skyblue", label="Clase:1 (Bueno)")
plt.scatter(malo["edad"], malo["credito"], marker="*", s=150, color="red", label="Clase:0 (Malo)")

plt.ylabel("Monto del Credito")
plt.xlabel("Edad")
plt.legend(bbox_to_anchor=(1, 0.2))
plt.show()

## Preparacion de los datos

In [ ]:
datos = data[["edad","credito"]]
clase = data["cumplio"]

escalador = preprocessing.MinMaxScaler()

datos = escalador.fit_transform(datos)

## Creacion del modelo KNN

In [ ]:
# Nuevo solicitante
edad = 40
monto = 350000
solicitante = escalador.transform([[edad, monto]])


metricas = [
    ('euclidean', {}),
    ('manhattan', {}),
    ('minkowski', {'p': 3}),
    ('hamming', {})
]

n_neigbors =[ 1, 3, 5, 7, 10]

# Datos sintéticos para graficar
creditos = np.array([np.arange(100000, 600010, 1000)] * 43).reshape(1, -1)
edades = np.array([np.arange(18, 61)] * 501).reshape(1, -1)
todos = pd.DataFrame(np.stack((edades, creditos), axis=2)[0], columns=["edad", "credito"])
solicitantes = escalador.transform(todos)

resultados =[]
for name, params in metricas:
    for k in n_neigbors:
        print(f"\n🔍 Distancia: {name} | Vecinos: {k}")

        # Entrenar clasificador
        clasificador = KNeighborsClassifier(n_neighbors=k, metric=name, **params)
        clasificador.fit(datos, clase)

        # Predicción del solicitante
        clase_predicha = clasificador.predict(solicitante)[0]
        probas = clasificador.predict_proba(solicitante)[0]
        print(f"Clase del solicitante: {clase_predicha}")
        print(f"Probabilidades: {probas}")

        # Predecir clases para todos los solicitantes
        clases_resultantes = clasificador.predict(solicitantes)
        bueno = todos[clases_resultantes == 1]
        malo = todos[clases_resultantes == 0]

        # Matriz de confusión y precisión
        y_pred = clasificador.predict(datos)
        matriz = confusion_matrix(clase, y_pred)
        acc = accuracy_score(clase, y_pred)
        print("📊 Matriz de confusión:")
        print(matriz)
        print(f"✅ Precisión: {acc:.4f}")

        resultados.append({
            "distancia": name,
            "vecinos": k,
            "precision": acc
        })

        # Graficar
        plt.figure(figsize=(8, 6))
        plt.title(f"Distancia: {name} | Vecinos: {k}")
        plt.scatter(bueno["edad"], bueno["credito"], marker="*", s=150, color="skyblue", label="Clase:1 (Bueno)")
        plt.scatter(malo["edad"], malo["credito"], marker="*", s=150, color="red", label="Clase:0 (Malo)")
        plt.scatter(edad, monto, marker="P", s=250, color="green", label="Solicitante")
        plt.xlabel("Edad")
        plt.ylabel("Monto del Crédito")
        plt.legend(bbox_to_anchor=(1, 0.3))
        plt.tight_layout()
        plt.show()

# Evaluación final
mejor = max(resultados, key=lambda x: x["precision"])
print("\n🏆 Mejor combinación:")
print(f"Distancia: {mejor['distancia']} | Vecinos: {mejor['vecinos']} | Precisión: {mejor['precision']:.4f}")